In [ ]:
%reload_kedro

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from datetime import timedelta
import pandas as pd
from crypto_thesis.utils import build_log_return
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Default settings

In [ ]:
pd.set_option("display.float_format", lambda x: "%.6f" % x)

### Base data

In [ ]:
min_years_existence = catalog.load("params:min_years_existence")
end_date = catalog.load("params:raw_binance_get_data.end_date")

In [ ]:
df_prm = catalog.load("prm_binance")
df_prm.loc[:, "date"] = df_prm["open_time"].dt.date.apply(str)
df_prm = df_prm[["open_time", "date", "close", "symbol"]]
df_prm = df_prm.sort_values(by=["symbol", "open_time"])

In [ ]:
df_prm_logret = df_prm.groupby("symbol").apply(build_log_return)
assert df_prm_logret["log_return"].isna().sum() == df_prm["symbol"].nunique()

In [ ]:
end_date = pd.to_datetime(end_date)
cutoff_3y = str((end_date - timedelta(days = min_years_existence * 365)).date())
end_date = str(end_date.date())

In [ ]:
df_prm_logret = df_prm_logret[df_prm_logret["date"] <= end_date]

In [ ]:
df_prm_logret.head()

### EDA: coins universe subset

In [ ]:
df_prm_grp = df_prm_logret.groupby("symbol").agg({"date": ["min", "max"]}).reset_index()

In [ ]:
df_prm_grp.columns = ["symbol", "date_min", "date_max"]

In [ ]:
df_prm_grp = df_prm_grp.sort_values(by="date_min", ascending=True)

In [ ]:
df_aux = df_prm_grp[(df_prm_grp["date_min"] >= cutoff_3y) | (df_prm_grp["date_max"] != end_date)]
symbols_to_delete = df_aux["symbol"].tolist()

In [ ]:
df_prm_grp_drop = df_prm_grp.drop(df_aux.index)

In [ ]:
print(f"Remaining symbols: {df_prm_grp_drop['symbol'].unique().tolist()}")
print(f"Remaining symbols number: {df_prm_grp_drop.shape[0]}")

In [ ]:
df_prm_grp_drop.head()

In [ ]:
# min and max values for Tron
df_prm[df_prm["symbol"] == "TRXUSDT"].agg({"close": ["min", "max"]})

### EDA: tabela estatística descritiva

In [ ]:
df_prm_dropped = df_prm[~df_prm["symbol"].isin(symbols_to_delete)]
assert df_prm_dropped.symbol.nunique() == 16

df_prm_dropped.loc[:, "symbol"] = df_prm_dropped["symbol"].str[:3]

In [ ]:
df_prm_logret = df_prm_dropped.groupby("symbol").apply(build_log_return)
assert df_prm_logret["log_return"].isna().sum() == df_prm_dropped["symbol"].nunique()

df_prm_logret.loc[:, "log_return"] = df_prm_logret["log_return"].fillna(0)

In [ ]:
df_prm_logret_drop = df_prm_logret[["open_time", "symbol", "log_return"]]

df_pivot = df_prm_logret.pivot(index="open_time", columns=["symbol"], values=["log_return"]) \
                        .fillna(0) \
                        .sort_index()

In [ ]:
df_desc = df_pivot.describe() \
                .unstack() \
                .reset_index() \
                .drop(columns=["level_0"]) \
                .rename(columns={"level_2": "metric",
                                0: "value"})

In [ ]:
df_pivot = df_desc.pivot(index=["symbol"], columns=["metric"])["value"]

df_pivot = df_pivot.reset_index()
df_pivot = df_pivot.drop(columns=["count"])

df_pivot = df_pivot[["symbol", "min", "25%", "50%", "75%", "max", "mean", "std"]]
df_pivot = df_pivot.sort_values(by="symbol")

In [ ]:
df_sixsigma = df_pivot.copy()

df_sixsigma.loc[:, "low_sixSigma"] = df_sixsigma["mean"] - 6*df_sixsigma["std"]
df_sixsigma.loc[:, "high_sixSigma"] = df_sixsigma["mean"] + 6*df_sixsigma["std"]

df_sixsigma.loc[:, "flag"] = df_sixsigma.apply(lambda col: True if col["min"] < col["low_sixSigma"] and \
                                                                    col["max"] > col["high_sixSigma"] \
                                                        else False, axis=1)

df_sixsigma

: 

In [ ]:
print(df_pivot.to_latex(index=False))

### EDA: cumulative log returns and percentage change

In [ ]:
df_prm_dropped = df_prm[~df_prm["symbol"].isin(symbols_to_delete)]
assert df_prm_dropped.symbol.nunique() == 16

In [ ]:
target_time_filter = df_prm_dropped.groupby("symbol")["open_time"].min().max()
df_prm_dropped = df_prm_dropped[df_prm_dropped["open_time"] >= target_time_filter]

df_prm_dropped.loc[:, "symbol"] = df_prm_dropped["symbol"].str[:3]

assert df_prm_dropped.open_time.min() == target_time_filter

In [ ]:
df_prm_logret = df_prm_dropped.groupby("symbol").apply(build_log_return)
assert df_prm_logret["log_return"].isna().sum() == df_prm_dropped["symbol"].nunique()

df_prm_logret.loc[:, "log_return"] = df_prm_logret["log_return"].fillna(0)
df_prm_logret.loc[:, "logret_cumsum"] = df_prm_logret.groupby("symbol")["log_return"].cumsum()

In [ ]:
df_pivot = df_prm_logret.pivot(index="open_time", columns=["symbol"], values=["logret_cumsum"]) \
                            .dropna().sort_index()

In [ ]:
# find negative and positive cumulative returns
df_aux = df_pivot["logret_cumsum"].copy().sort_index()
df_aux = df_aux.applymap(lambda row: np.exp(row) - 1)
df_tail = df_aux.tail(1)

df_tail_negative = df_tail[df_tail < 0]
df_logret_negative = pd.melt(df_tail_negative).dropna().rename(columns={"variable": "symbol", "value": "pctchg"})

print(df_logret_negative)
print()

df_tail_positive = df_tail[df_tail > 0]
df_logret_positive = pd.melt(df_tail_positive).dropna().rename(columns={"variable": "symbol", "value": "pctchg"})

print(df_logret_positive.sort_values(by="pctchg", ascending=False))

In [ ]:
df_aux = df_pivot["logret_cumsum"].copy().sort_index()
plot = df_aux.plot(figsize=(15, 8))
plot.legend(bbox_to_anchor=(1.0, 1.015))
plot.set_xlabel("Date")
plot.set_ylabel("Cumulative LogRets")

fig = plot.get_figure()
fig.savefig("/Users/gpalazzo/Desktop/fig_cum_logrets_cryptos.png")

In [ ]:
# ***** esse plot não ficou legal, manter o de log retornos *****

# df_aux = df_pivot["pctchg_cumsum"].copy()
# plot = df_aux.plot(legend=None, figsize=(15, 8))
# plot.set_xlabel("Date")
# plot.set_ylabel("Cumulative pctChg")

# fig = plot.get_figure()
# fig.savefig("/Users/gpalazzo/Desktop/fig_cum_pctchg_cryptos.png")

### EDA: log returns distribution

In [ ]:
df_prm_dropped = df_prm_logret[~df_prm_logret["symbol"].isin(symbols_to_delete)]

for symbol in df_prm_dropped["symbol"].unique():

    print(f"Building plot for: {symbol}")
    
    df_aux = df_prm_dropped[df_prm_dropped["symbol"] == symbol][["open_time", "log_return"]]
    df_aux = df_aux.set_index("open_time").sort_index()
    
    plt.figure(figsize=(10,4))
    plt.title(symbol)
    sns.distplot(df_aux["log_return"])

### EDA: boxplot

In [ ]:
df_prm_dropped = df_prm_logret[~df_prm_logret["symbol"].isin(symbols_to_delete)]

df_prm_dropped.loc[:, "symbol"] = df_prm_dropped["symbol"].str[:3]

In [ ]:
df_pivot = df_prm_dropped.pivot(index="open_time", columns=["symbol"], values=["log_return"]).dropna()
df_pivot = df_pivot.droplevel(level=0, axis=1)

symbols = df_pivot.columns.tolist()
df_pivot.columns = symbols

In [ ]:
symbol_std_descending = df_pivot.std().reset_index().rename(columns={"index": "symbol", 0: "std"}) \
    .sort_values(by="std", ascending=False) \
    ["symbol"].tolist()

df_pivot = df_pivot[symbol_std_descending]

In [ ]:
plt.figure(figsize=(15, 8))
plot = sns.boxplot(df_pivot)

plot.set_xlabel("Symbols")
plot.set_ylabel("LogRets")
plt.xticks(rotation = 30)

fig = plot.get_figure()
fig.savefig("/Users/gpalazzo/Desktop/fig_box_plot_cryptos.png")

### EDA: corr matrix

In [ ]:
df_prm_dropped = df_prm_logret[~df_prm_logret["symbol"].isin(symbols_to_delete)]

df_prm_dropped.loc[:, "symbol"] = df_prm_dropped["symbol"].str[:3]

df_pivot = df_prm_dropped.pivot(index="open_time", columns=["symbol"], values=["log_return"]).dropna()
df_pivot = df_pivot.droplevel(level=0, axis=1)

symbols = df_pivot.columns.tolist()
df_pivot.columns = symbols

In [ ]:
corr_mtx = round(df_pivot.corr(), 2)
mask = np.triu(np.ones_like(corr_mtx, dtype=bool))

In [ ]:
# find min and max correlation pairs
corr_mtx_parsed = corr_mtx[corr_mtx > 0].unstack().reset_index().dropna().rename(columns={0: "value"})
corr_mtx_parsed = corr_mtx_parsed[corr_mtx_parsed["value"] < 1]
corr_mtx_parsed.sort_values(by="value")

In [ ]:
plt.figure(figsize=(15, 9))

plot = sns.heatmap(corr_mtx, 
                   annot=True, 
                   mask=mask, 
                   center=0,
                   square=True, 
                   linewidths=.5, 
                   cbar_kws={"shrink": .5}
                  )

plt.yticks(rotation=0)

fig = plot.get_figure()
fig.savefig("/Users/gpalazzo/Desktop/fig_corr_mtx_cryptos.png")

### EDA: captured variance

# REVISAR ESSA SEÇÃO PORQUE ESTÁ MEIO GAMBIARRA!

In [ ]:
df_prm_dropped = df_prm_logret[~df_prm_logret["symbol"].isin(symbols_to_delete)]
df_prm_dropped = df_prm_dropped.sort_values(by=["symbol", "open_time"])

In [ ]:
df_fte = catalog.load("fte_binance")
df_window_nbr = catalog.load("window_nbr_lookup_multic")

In [ ]:
# get lower window durations
df_fte_aux = df_fte.dropna()

df_fte_aux = df_fte_aux[df_fte_aux["window_duration_sec"] != 0.0]
_min_duration = df_fte_aux.window_duration_sec.min()
df_fte_aux = df_fte_aux[df_fte_aux["window_duration_sec"].between(_min_duration, _min_duration + 4500)]

In [ ]:
df_fte_window = df_fte_aux.merge(df_window_nbr[["open_time", "close_time", "window_nbr"]]\
                                 , on=["open_time", "close_time"], how="inner")

df_fte_window = df_fte_window[["open_time", "close_time", "window_nbr"] + \
                             [col for col in df_fte_window.columns if col \
                                  not in ["open_time", "close_time", "window_nbr"]
                             ]]

df_fte_window = df_fte_window[df_fte_window["window_nbr"].isin([1671, 1672, 1673, 1674, 1675])]

In [ ]:
possible_coins = list(set([col.split("__")[-1] \
     for col in df_fte_window.set_index(["open_time", "close_time", "window_nbr"]).columns \
     if col != "window_duration_sec"]))

possible_coins

In [ ]:
feature_example = "ATOMUSDT"

In [ ]:
df_fte_window[[col for col in df_fte_window.columns if col.endswith(feature_example)]].columns

In [ ]:
df_fte_aux = df_fte_window.reset_index(drop=True).tail(10) #get only the last window
df_fte_aux = df_fte_aux[
                        ["open_time", "close_time"] \
                        + [col for col in df_fte_aux.columns if col.endswith(feature_example)]
                       ]
df_fte_aux.head()

In [ ]:
df_prm_target = df_prm_dropped[df_prm_dropped["symbol"] == feature_example]

_open_time = df_fte_aux["open_time"].min()
_close_time = df_fte_aux["close_time"].max()

df_prm_target = df_prm_target[df_prm_target["open_time"].between(_open_time, _close_time)]
df_prm_target = df_prm_target[["open_time", "log_return"]]

df_prm_target.head()

In [ ]:
df_fte_aux = df_fte_aux.drop(columns=["open_time"])
df_prm_target = df_prm_target.rename(columns={"open_time": "close_time"})

final_df = df_prm_target.merge(df_fte_aux, on="close_time", how="left")

final_df.head()

In [ ]:
final_df = final_df.ffill().bfill()

In [ ]:
final_df.corr()["log_return"].sort_values(ascending=False)

In [ ]:
# zscore__ETHUSDT              0.457699
# zscore__XRPUSDT              0.184618
# log_return_std__LTCUSDT      0.183035
# zscore__ETHUSDT              0.179166
# zscore__ADAUSDT              0.167575